In [1]:
#import dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name

In [2]:
# Create Engine and Pass in MySQL Connection

# local Postgres DB Connection  
local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")
#print(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Create a local database engine connection
local_conn = local_engine.connect()

# Create local database
try:
    local_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {local_db_name}")
except Exception as e:
    print(e)

# Connect to local database
try:
    local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}/{local_db_name}")
    local_conn = local_engine.connect()
except Exception as e:
    print(e)  
    
#confirm tables
local_engine.table_names()

(psycopg2.errors.DuplicateDatabase) database "project2_db" already exists

[SQL: CREATE DATABASE project2_db]
(Background on this error at: http://sqlalche.me/e/13/f405)


['2020_fantasy_stats']

In [3]:
def load_rows():
    # load 300 rows view
    load_300 = browser.links.find_by_partial_text('300')
    load_300.click()

    # load more rows
    load_more = browser.links.find_by_partial_text('Load More')
    load_more[1].click()
    # for x in range(6):
    #     load_more[1].click()

In [4]:
def stats():
    # save the stats table as a df
    stats_table = fantasy_soup.find("table", id="stats_grid")
    stats_df = pd.read_html(f'<table>${stats_table.tbody}</table>')[0]
    return stats_df

In [5]:
def names():
    # save the names table as a df
    table_names = fantasy_soup.find("div", class_="k-grid-content-locked")
    names_df = pd.read_html(f'<table>${table_names.tbody}</table>')[0]
    return names_df

In [6]:
def combine():
    col_names = ['RK', 'NAME', 'TEAM', 'POS', 'GMS', 'PASS YDS', 'PASS TD', 'PASS INT', 'RUSH YDS', 'RUSH TD', 'REC', 'REC YDS', 'REC TD', 'DEF SCK', 'DEF INT', 'DEF FF', 'DEF FR', 'FPTS/G', 'FPTS']
    combined_df = pd.concat([names_df, stats_df], axis=1)
    combined_df.columns = [col_names]
    return combined_df

In [7]:
def to_db():
    #upload dataframe to database
    #combined_df.to_sql(name=f'{season}_fantasy_stats', if_exists='replace', con=local_conn, index=False)
    stats_df.to_sql(name=f'{season}_fantasy_stats', if_exists='replace', con=local_conn, index=False)
    names_df.to_sql(name=f'{season}_fantasy_stats', if_exists='replace', con=local_conn, index=False)

In [8]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\Steven\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [9]:
# URL of page to be scraped

#for season in range(2010, 20210):
season = '2020'
url = (f'https://fantasydata.com/nfl/fantasy-football-leaders?season={season}&seasontype=1&scope=1&subscope=1&startweek=1&endweek=1&aggregatescope=1&range=1')
url

# Retrieve page with the browser module
browser.visit(url)

In [10]:
# setup html parser
html = browser.html
fantasy_soup = BeautifulSoup(html, 'html.parser')

#load_rows()
stats()
names()
combine()
to_db()

NameError: name 'names_df' is not defined